In [53]:
%pylab notebook
from time import sleep
from spi_helper import *
sys.path.append("../..")
from common import *

Populating the interactive namespace from numpy and matplotlib


# Connec to FPGA

Start remote litex server first
```bash
make server
```

In [54]:
r = conLitexServer('../build/csr.csv')

Connected to Port 1234
AD9174 + VC707 test 2020-04-19 23:17:22


# Play with GTX phy

In [1045]:
# Resets sys and jesd clock domain
r.regs.ctrl_reset.write(1)
r.regs.ctrl_reset.read()

0

### jesd clock frequency (160 MHz!)

In [1046]:
print('{:.6f} MHz'.format(r.regs.crg_f_jesd_value.read() / 1e6))

160.000384 MHz


In [1163]:
# Triggers the GTXInit state machine
r.regs.control_enable.write(1)
r.regs.control_phy_done.read()

1

In [1242]:
# Transmit a square wave of frequency line_rate / 40
on = 1
for i in range(4):
    getattr(r.regs, 'phy{}_transmitter_produce_square_wave'.format(i)).write(on)

In [790]:
# Transmit pseudo random test pattern
# 0 = off, 1 = PRBS7 (5, 6), 2 = PRBS15 (13, 14), 3 = PRBS31 (27, 30)
r.regs.control_prbs_config.write(0)
r.regs.control_prbs_config.read()

0

In [208]:
r.regs.control_stpl_enable.write(0)
r.regs.control_stpl_enable.read()

0

In [1164]:
# AD9174 --> to JESD204BLinkTX, triggers ILAS after CGS
r.regs.control_jsync.read()

0

In [1165]:
r.regs.control_ready.read()

0

# Setup AD9174

In [55]:
ad = AdSpi(r)

reg PLL_STATUS (7b5) already exist (281)


## General init

In [1157]:
# Power up sequence, Table 51
ad.wr(0x000, 0x81)  # Soft reset
ad.wr(0x000, 0x3C)  # 4 - wire SPI mode + ADDRINC

ad.wr(0x091, 0x00)  # Power up clock RX
ad.wr(0x206, 0x01)  # Enable PHYs
ad.wr(0x705, 0x01)  # LOAD NVRAM FACTORY SETTINGS
ad.wr(0x090, 0x00)  # Power on DACs and bias supply
print('AD917X_NVM_BLR_DONE:', (ad.rr(0x705) >> 1) & 1)

# Print product ID (0x9174)
val = ad.rr('SPI_PRODIDH') << 8 | ad.rr('SPI_PRODIDL')
print('PROD_ID: 0x{:04x}'.format(val))
val = ad.rr('SPI_CHIPGRADE')
print('PROD_GRADE: {:x}  DEV_REVISION: {:x}'.format(val >> 8, val & 0xF))

AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5


In [1158]:
ad.wr(0x100, 0x01)  # Put digital datapath in reset

# Disable DAC PLL and config for external clock, Table 52
ad.wr(0x095, 0x01)
ad.wr(0x790, 0xFF)  # DACPLL_POWER_DOWN
ad.wr(0x791, 0x1F)

# ADC clock output divider = /4
#   0: Divide by 1
#   1: Divide by 2
#   2: Divide by 3
#   3: Divide by 4
BIT_ADC_CLK_DIVIDER = 6
ad.wr(0x799, (3 << BIT_ADC_CLK_DIVIDER) | 8)

# Delay Lock Loop (DLL) Configuration
ad.wr(0x0C0, 0x00)  # Power-up delay line.
ad.wr(0x0DB, 0x00)  # Update DLL settings to circuitry.
ad.wr(0x0DB, 0x01)
ad.wr(0x0DB, 0x00)
ad.wr(0x0C1, 0x68)  # set search mode for f_DAC > 4.5 GHz
ad.wr(0x0C1, 0x69)  # set DLL_ENABLE
ad.wr(0x0C7, 0x01)  # Enable DLL read status.
print('DLL locked:', ad.rr(0x0C3))

ad.wr(0x008, (0b01 << 6) | 0b000001)  # Select first DAC and first channel
print('SPI_PAGEINDX: 0b{:08b}'.format(ad.rr('SPI_PAGEINDX')))

# Magic numbers from Table 54 (calibration)
ad.wr(0x050, 0x2A)
ad.wr(0x061, 0x68)
ad.wr(0x051, 0x82)
ad.wr(0x051, 0x83)
print('CAL_STAT:', ad.rr(0x052))  # 1 = success
ad.wr(0x081, 0x03)  # Power down calibration clocks

DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1


0

## JESD init

on `AD9172_FMC_EBZ` GT lanes 0 - 3 are of __inverted__ polarity!
But this has been taken into account in the bit-file.

In [1159]:
L = 4
M = 2
F = 4
S = 1
NP = 16
N = 16
data_rate = 5.12e9 / 8 / 4
lane_rate = M / L * NP * 10 / 8 * data_rate
print('data rate {} MSps,  lane rate {} MBps'.format(data_rate / 1e6, lane_rate / 1e6))

# JESD config, Table 55
ad.wr(0x201, 0x00)  # Power down no PHYs
ad.wr(0x100, 0x00)  # Power up digital datapath clocks
ad.wr(0x110, (0 << 5) | 0)  # JESD_MODE: Single link, mode 0

# Mode 0 (L = 1, M = 2, F = 4, S = 1, NP = 16, N = 16)
# 1 lane, 2 converters (I0, Q0), 4 byte / frame, 1 sample / frame, 16 bit 
# 32 bit / frame = 1 sample, 128 MSps from FPGA, DAC at 4.096 GSps? 

ad.wr(0x111, 0x84)   # DP_INTERP_MODE = 8x, CH_INTERP_MODE = 4x
print('MODE_NOT_IN_TABLE:', (ad.rr(0x110) >> 7) & 0x01)

ad.wr(0x084, 1)  # SYSREF_PD: Power down sysref reciever, we run subclass 0
ad.wr(0x300, 0b0001)  # select single link, page link0, enable link0
ad.wr(0x475, 0x09)  # Soft reset the JESD204B quad-byte deframer
ad.wr(0x453, (0 << 7) | (L - 1))  # disable scrambler, L-value - 1
ad.wr(0x454, (F - 1))
ad.wr(0x456, (M - 1))
ad.wr(0x457, (N - 1))
ad.wr(0x458, (0 << 5) | (NP - 1)) # subclass 0, NP-value - 1
ad.wr(0x459, (S - 1)) # Version = JESD204A, samples per converter per frame cycle (minus 1)

ad.wr(0x475, 1)  # Bring the JESD204B quad-byte deframer out of reset.

# Table 56: skipped as Channel interpolation mode = 1x

data rate 160.0 MSps,  lane rate 1600.0 MBps
MODE_NOT_IN_TABLE: 0


0

In [650]:
ad.help(0x280)

reg 0x280, PLL_ENABLE_CTRL:
    bit 7:3, RESERVED, R, reset 0x00
    Reserved.

    bit 2, LOLSTICKYCLEAR_LCPLL_RC, R/W, reset 0x00
    Clears out loss of lock bit.

    bit 1, LDSYNTH_LCPLL_RC, R/W, reset 0x00
    Pulse high to start VCO calibration (without restarting the regulator or remeasuring the temperature).

    bit 0, SERDES_PLL_STARTUP, R/W, reset 0x01
    SERDES circuitry blocks are powered off when this bit is set to 0. Set this bit to 1 at the end of the SERDES configuration writes. When this bit is set to 1, it powers up the SERDES PLL blocks and starts the LDO and calibration routine to lock the PLL auto- matically to the appropriate lane rate based on the JESD204B mode and interpolation options programmed in the device. The SERDES_PLL_LOCK bit (Register 0x281, Bit 0) reads 1 when the PLL achieves lock.



In [1205]:
ad.wr(0x280, 0x00)  # enable serdes PLL

# EQ settings for < 11 dB insertion loss
ad.wr(0x240, 0xAA)
ad.wr(0x241, 0xAA)
ad.wr(0x242, 0x55)
ad.wr(0x243, 0x55)
ad.wr(0x244, 0x1F)
ad.wr(0x245, 0x1F)
ad.wr(0x246, 0x1F)
ad.wr(0x247, 0x1F)
ad.wr(0x248, 0x1F)
ad.wr(0x249, 0x1F)
ad.wr(0x24A, 0x1F)
ad.wr(0x24B, 0x1F)

ad.wr(0x201, 0x00)
# ad.wr(0x201, 0xF0)  # Power down the unused upper 4 PHYs

# ad.wr(0x203, 0x01)  # Power down sync1
# ad.wr(0x253, 0x01)  # Sync0 = LVDS
# ad.wr(0x254, 0x01)  # Sync1 = LVDS

# SERDES required register write.
ad.wr(0x200, 0x01)  # Power down the SERDES blocks.
ad.wr(0x210, 0x16)
ad.wr(0x216, 0x05)
ad.wr(0x212, 0xFF)
ad.wr(0x212, 0x00)
ad.wr(0x210, 0x87)
ad.wr(0x210, 0x87)
ad.wr(0x216, 0x11)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x200, 0x00)  # Power up the SERDES circuitry blocks.
sleep(0.1)

# SERDES required register write.
ad.wr(0x210, 0x86)
ad.wr(0x216, 0x40)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x210, 0x86)
ad.wr(0x216, 0x00)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x210, 0x87)
ad.wr(0x216, 0x01)
ad.wr(0x213, 0x01)
ad.wr(0x213, 0x00)
ad.wr(0x280, 0x05)
# Start up SERDES PLL circuitry blocks and initiate SERDES PLL calibration.
ad.wr(0x280, 0x01)
print('SERDES PLL locked:', ad.rr(0x281) & 0x01)

# ad.wr(0x03B, 0xF1)  # Enable the sync logic, and set the rotation mode to reset the synchronization logic upon a sync reset trigger.
# ad.wr(0x03A, 0x02)  # Set up sync for one-shot sync mode.
# ad.wr(0x300, 0b0001)  # select single link, page link0, enable link0

SERDES PLL locked: 1


# PHY snapshot

In [ ]:
r.regs.phy0_transmitter_tp_on

In [57]:
# Transmit a square wave of frequency line_rate / 40
on = 1
for i in range(8):
    getattr(r.regs, 'phy{}_transmitter_tp_on'.format(i)).write(on)

In [1418]:
# Transmit pseudo random test pattern
# 0 = off, 1 = PRBS7 (5, 6), 2 = PRBS15 (13, 14), 3 = PRBS31 (27, 30)
r.regs.control_prbs_config.write(0)
r.regs.control_prbs_config.read()

0

In [1407]:
# generate a PRBS7 bit-pattern to compare against
a = 2
for i in range(255):
    nb = (((a >> 6) ^ (a >> 5)) & 1)
    a = ((a << 1) | nb)
prbs7_str = "{:b}".format(a)
prbs7_str_inv = "{:b}".format(a ^ (2**256 - 1))

In [65]:
ad.wr('PHY_PRBS_TEST_EN', 0xFF)  # Needed: clock to test module
ad.wr('PHY_PRBS_TEST_CTRL', 0b01)  # rst

print('PHY_SNAPSHOT_DATA:')
for lane in range(8):
    ad.wr('PHY_PRBS_TEST_CTRL', (lane << 4))    
    ad.wr('PHY_DATA_SNAPSHOT_CTRL', 0x01)
    ad.wr('PHY_DATA_SNAPSHOT_CTRL', 0x00)
    val = 0
    for i in range(5):
        val = (val << 8) | ad.rr(0x323 - i)
    bVal = '{:040b}'.format(val)
    desc = ''
    if bVal.find('00111110101100000101') >= 0:
        desc = ','
    elif prbs7_str.find(bVal) >= 0:
        desc = 'PRBS7'
    elif prbs7_str_inv.find(bVal) >= 0:
        desc = '/PRBS7'
    print('{0:}: 0x{1:010x}, 0b{2:}, {3:}'.format(lane, val, bVal, desc))

PHY_SNAPSHOT_DATA:
0: 0x0000fffff0, 0b0000000000000000111111111111111111110000, 
1: 0x7ffff00000, 0b0111111111111111111100000000000000000000, 
2: 0x7ffff80000, 0b0111111111111111111110000000000000000000, 
3: 0xfff800007f, 0b1111111111111000000000000000000001111111, 
4: 0xff800007ff, 0b1111111110000000000000000000011111111111, 
5: 0xffff00000f, 0b1111111111111111000000000000000000001111, 
6: 0x0001ffffe0, 0b0000000000000001111111111111111111100000, 
7: 0xfe00001fff, 0b1111111000000000000000000001111111111111, 


Lane 4 - 7 seem to show commas sometimes!

`K28.5` = 0011111010 / 1100000101 

# PRBS check

In [1422]:
prbs = 0  # 0: PRBS7, 1: PRBS15, 2: PRBS31
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)
ad.wr('PHY_PRBS_TEST_EN', 0xFF)  # Enable test for all lanes
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b01)  # rst
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)
ad.wr('PHY_PRBS_TEST_THRESHOLD_LOBITS', 0xFF)  # error threshold

ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b10)  # start test
sleep(0.01)
ad.wr('PHY_PRBS_TEST_CTRL', (prbs << 2) | 0b00)  # stop test

print('PHY_PRBS_TEST_STATUS: {:08b} (high = pass)'.format(ad.rr('PHY_PRBS_TEST_STATUS')))
for lane in range(8):
    ad.wr('PHY_PRBS_TEST_CTRL', (lane << 4) | (prbs << 2) | 0b00)  # read-out lane X
    err_cnt = ad.rr('PHY_PRBS_TEST_ERRCNT_HIBITS') << 16 | \
              ad.rr('PHY_PRBS_TEST_ERRCNT_MIDBITS') << 8 | \
              ad.rr('PHY_PRBS_TEST_ERRCNT_LOBITS')
    print('lane: {}, err_cnt: {:06x}'.format(lane, err_cnt))

PHY_PRBS_TEST_STATUS: 10000000 (high = pass)
lane: 0, err_cnt: 88b5e4
lane: 1, err_cnt: 88b5e4
lane: 2, err_cnt: 88b5e4
lane: 3, err_cnt: 88b5e4
lane: 4, err_cnt: 88b5e4
lane: 5, err_cnt: 88b5e4
lane: 6, err_cnt: 88b5e4
lane: 7, err_cnt: 000000


# Setup for internal PRBS check

In [1420]:
ad.wr(0x240, 0)
ad.wr(0x241, 0)
# ad.wr('CDR_BITINVERSE', 0x00)
ad.wr('LBT_REG_CNTRL_0', 0xFF)  # enable loopback for lane X
halfrate = 0  # 1: enable halfrate mode (keep off!)
ad.wr('LBT_REG_CNTRL_1', (halfrate << 1) | 1)
ad.wr('LBT_REG_CNTRL_1', halfrate << 1)

0

In [338]:
ad.rr(0x201)

0

# Setup HMC7044
  * External VCO input
  * clkout12 = f_vco / 8 = 160 MHz

In [1038]:
hmc = HmcSpi(r)

hmc.wr(0x000, 1)  # reset
hmc.wr(0x000, 0)

hmc.wr(0x054, 0)  # Disable SDATA driver (uni-direct. buffer)
hmc.wr(0x001, (1 << 6) | (1 << 5)) # High performance dividers / PLL

# VCO Selection
# 0 Internal disabled/external
# 1 High
# 2 Low
VCO_SELECT = 3
hmc.wr(0x003, (0 << VCO_SELECT))

# Enable output channel 12 and 13
hmc.wr(0x004, (1 << (12 // 2)))

# clkin1 as external VCO input
hmc.wr(0x005, (1 << 5))

# Magic numbers from datasheet Table 74
hmc.wr(0x09F, 0x4D)
hmc.wr(0x0A0, 0xDF)
hmc.wr(0x0A5, 0x06)
hmc.wr(0x0A8, 0x06)
hmc.wr(0x0B0, 0x04)

hmc.setupChannel(12, 8)  # DEV_CLK = 160 MHz
hmc.setupChannel(13, 80)  # SYSREF2 = 16 MHz

### Check jesd clock at FPGA (160 MHz)

In [1032]:
r.regs.crg_f_jesd_value.read()

213333849

In [ ]:
r.close()